In [1]:
from __future__ import annotations

# Import the ChatGroq class from the langchain_groq package
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from typing import TypedDict, List, Annotated
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
class Task(BaseModel):
    id: str
    title: str
    brief: str = Field(description="What to Cover in the blog post")

In [4]:
class Plan(BaseModel):
    blog_title: str
    tasks: List[Task]
    

In [5]:
import operator

class State(TypedDict):
    topic: str
    plan: Plan
    # task: Task  # for individual worker execution
    sections: Annotated[List[str], operator.add]
    final: str

In [6]:
llm = ChatGroq(model="llama-3.3-70b-versatile")


In [7]:
llm.invoke("Good Morning")

AIMessage(content='Good morning! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 37, 'total_tokens': 48, 'completion_time': 0.013930392, 'completion_tokens_details': None, 'prompt_time': 0.003229794, 'prompt_tokens_details': None, 'queue_time': 0.168246926, 'total_time': 0.017160186}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_45180df409', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c2e21-2608-7d10-9b5c-b88527c645d7-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 11, 'total_tokens': 48})

In [8]:
def orchestrator(state: State):
    plan = llm.with_structured_output(Plan).invoke(
        [
            SystemMessage(
                content=(
                    "You are an experienced technical writer who creates clear, friendly, and engaging blog plans. "
                    "Your audience includes recruiters, hiring managers, and professionals who prefer concise, readable content. "
                    "Avoid jargon-heavy language. Focus on clarity, real-world relevance, and logical flow."
                )
            ),
            HumanMessage(
                content=(
                    f"Create a blog outline with 6–7 well-structured sections on the topic below.\n\n"
                    f"Topic: {state['topic']}\n\n"
                    "Each section should:\n"
                    "- Have a clear, engaging title\n"
                    "- Include a short 1–2 sentence description\n"
                    "- Be written in a friendly, conversational tone\n"
                    "- Emphasize practical insights and real-world value\n\n"
                    "The outline should feel approachable, professional, and easy to skim."
                )
            )
        ]
    )
    return {"plan":plan}
    

In [9]:
# Visualize(orchestrator)

In [10]:
plan = llm.with_structured_output(Plan).invoke(
    [
        SystemMessage(
            content=(
                "You are an experienced technical writer who creates clear, friendly, and engaging blog plans. "
                "Your audience includes recruiters, hiring managers, and professionals who prefer concise, readable content. "
                "Avoid jargon-heavy language. Focus on clarity, real-world relevance, and logical flow."
            )
        ),
        HumanMessage(
            content=(
                f"Create a blog outline with 6–7 well-structured sections on the topic below.\n\n"
                f"Topic: agentic AI \n\n"
                "Each section should:\n"
                "- Have a clear, engaging title\n"
                "- Include a short 1–2 sentence description\n"
                "- Be written in a friendly, conversational tone\n"
                "- Emphasize practical insights and real-world value\n\n"
                "The outline should feel approachable, professional, and easy to skim."
            )
        )
    ]
)



In [11]:
plan.blog_title

'Unlocking the Power of Agentic AI: A Guide to Smarter Automation'

In [12]:
plan.tasks

[Task(id='1', title='Introduction to Agentic AI', brief='Explore the concept of agentic AI and its potential to revolutionize automation, highlighting its benefits and applications in real-world scenarios.'),
 Task(id='2', title='Understanding Agency in AI', brief='Dive into the definition and implications of agency in AI, discussing how it enables machines to make decisions and take actions autonomously.'),
 Task(id='3', title='The Future of Work with Agentic AI', brief='Discuss the impact of agentic AI on the job market, including the creation of new job opportunities and the need for workers to develop new skills.'),
 Task(id='4', title='Agentic AI in Action: Real-World Examples', brief='Examine practical examples of agentic AI in various industries, such as healthcare, finance, and transportation, highlighting its potential to drive innovation and efficiency.'),
 Task(id='5', title='Overcoming Challenges and Concerns', brief='Address the challenges and concerns surrounding agentic 

In [13]:
def task_distributor(state: State):
    """
    Dynamically creates one worker per task.
    The number of workers equals the number of sections in the plan.
    """

    tasks = state["plan"].tasks

    # Create one Send event per task
    sends = [
        Send(
            "section_writer",  # name of the worker node
            {
                "topic": state["topic"],
                "task": task,  # pass individual task
                "plan": state["plan"],
            },
        )
        for task in tasks
    ]

    return sends

In [14]:
def worker(payload: dict):
    task = payload["task"]
    plan = payload["plan"]
    topic = payload["topic"]
    
    # write the section
    section_md = llm.invoke(
        [
            SystemMessage(
                content=(
                    "write one clean markdown section"
                )
            ),
            HumanMessage(
                content=(
                    f"Write a blog section on the topic below in markdown only.\n\n"
                    f"Blog: {plan.blog_title}\n\n"
                    f"Topic: {topic}\n\n"
                    f"Section: {task.title}\n\n"
                    f"Description: {task.brief}\n\n"
                )
            )
        ]
    ).content.strip()
    
    return {"sections": [section_md]}



In [15]:
from pathlib import Path


def finalizer(state: State):
    title = state["plan"].blog_title
    sections = state["sections"]
    final_md = f"# {title}\n\n" + "\n\n".join(sections)
    
    # save file
    file_name = title.lower().replace(" ", "_").replace(":", "").replace("'", "") + ".md"
    Path(file_name).write_text(final_md, encoding="utf-8")
    
    return {"final": final_md}


In [16]:
graph = StateGraph(State)


graph.add_node("orchestrator", orchestrator)
graph.add_node("section_writer", worker)
graph.add_node("finalizer", finalizer)

graph.add_edge(START, "orchestrator")
graph.add_conditional_edges(
    "orchestrator",
    task_distributor,
    ["section_writer"]
)

graph.add_edge("section_writer", "finalizer")
graph.add_edge("finalizer", END)

app = graph.compile()

In [17]:
blog = app.invoke({"topic": "langgraph","sections":[]})

In [18]:
blog

{'topic': 'langgraph',
 'plan': Plan(blog_title="Unlocking the Power of Langgraph: A Beginner's Guide", tasks=[Task(id='1', title='Introduction to Langgraph', brief='Discover what Langgraph is and its significance in the industry, with a brief overview of its applications and benefits.'), Task(id='2', title='Getting Started with Langgraph', brief='Learn how to set up Langgraph and start exploring its features, with step-by-step instructions and tips for beginners.'), Task(id='3', title='Langgraph in Action: Real-World Examples', brief='Explore real-world examples of Langgraph in use, highlighting its practical applications and the value it can bring to businesses and individuals.'), Task(id='4', title='Tips and Tricks for Mastering Langgraph', brief='Get expert advice on how to get the most out of Langgraph, including tips on optimization, customization, and integration with other tools.'), Task(id='5', title='Common Challenges and Solutions', brief='Address common challenges and pitfa